In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sec_edgar.data.preprocessing.preprocess_4form_files import pre_process_4form_archive_files
from sec_edgar.data.preprocessing.preprocess_master_idx import pre_process_master_idx_content
from sec_edgar.data.processing.process_4form_files import Process4FormFiles
from sec_edgar.data.processing.process_append_historical_prices import ProcessAppendHistoricalPrices
from sec_edgar.data.utils.cik_mapping_util import CikMappingUtil
from sec_edgar.stock_historical_data.av_historical_data_request import AVHistoricalDataRequest
import pandas as pd

from sec_edgar.utils.evaluation_metrics import PerformanceMetric01, PerformanceMetric02, PerformanceMetric03
from sec_edgar.utils.performance_evaluations import PerformanceEvaluations4Form, PerformanceEvaluations4FormYear

In [2]:
pd.set_option('display.width', 2500)      # max table width to display

In [3]:
## Loading of data set

In [4]:
data_set_name = "us_market_allderv_AandB_1019.py"
base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

path_processed_datasets = base_path + '/Data/processed_datasets/'
file_name = path_processed_datasets + data_set_name + ".csv"

processed_4form_df = pd.read_csv(file_name, index_col=0)

In [5]:
print(file_name)
display(processed_4form_df)
display(processed_4form_df.columns.to_list())

C:\Users\andre\OneDrive\Documents\MathModels\sec_edgar/Data/processed_datasets/us_market_allderv_AandB_1019.py.csv


,CIK,Company_Names,Date_Filed,index_id,index_id_min,n_transactions,shares_percent_changes_mean,transactionSharesAdjust,date_ft_delta_mean,Filename,...,Shifted Price pct_change (10)(10 days 00:00:00),Shifted Price pct_change (10)(-10 days +00:00:00),Shifted Price pct_change (15)(15 days 00:00:00),Shifted Price pct_change (15)(-15 days +00:00:00),Shifted Price pct_change (21)(21 days 00:00:00),Shifted Price pct_change (21)(-21 days +00:00:00),Shifted Price pct_change (31)(31 days 00:00:00),Shifted Price pct_change (31)(-31 days +00:00:00),Shifted Price pct_change (42)(42 days 00:00:00),Shifted Price pct_change (42)(-42 days +00:00:00)
index,,,,,,,,,,,,,,,,,,,,,
48441,27904,DELTA AIR LINES INC /DE/,2010-01-04,[140546],140546,1,NaN,0.00,0 days 00:00:00,['edgar/data/27904/0001019687-10-000008.txt'],...,-0.071374,0.086661,-0.049117,0.075472,-0.043511,0.187556,0.140489,0.352113,0.174694,0.701164
107523,935703,DOLLAR TREE INC,2010-01-04,[578414 578415 578416 578417 578418],578414,5,0.250942,3544.61,3 days 00:00:00,['edgar/data/935703/0001127602-10-000234.txt'\...,...,0.005482,0.000612,-0.034031,0.014925,0.143033,0.017983,0.211652,0.053858,0.232375,-0.000204
48728,28412,COMERICA INC /NEW/,2010-01-04,[79905 79906 79907 79908 79909 79910 79911 799...,79905,13,0.018748,1152.00,5 days 00:00:00,['edgar/data/28412/0000028412-10-000001.txt'\n...,...,0.011134,0.068302,0.030569,0.152212,0.014648,0.167237,0.143071,0.266620,0.203143,0.213538
75418,72741,NORTHEAST UTILITIES,2010-01-04,[378340 378341 378342 378343],378340,4,0.146730,9435.00,0 days 00:00:00,['edgar/data/72741/0000072741-10-000004.txt'\n...,...,-0.040546,-0.000762,-0.051874,0.003069,0.028191,0.022868,0.074148,0.055285,0.109829,0.049897
97444,882835,ROPER INDUSTRIES INC,2010-01-04,[229120 229121 229122 229123],229120,4,-0.027286,0.00,5 days 00:00:00,['edgar/data/882835/0001209191-10-000310.txt'],...,-0.046095,-0.010336,-0.029718,-0.028972,0.074132,-0.028318,0.083082,-0.027359,0.165974,-0.048330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61641,49071,HUMANA INC,2019-12-31,[795842 795844 795845 795846 795847 795848 795...,795842,14,NaN,0.00,0 days 00:00:00,['edgar/data/49071/0000049071-19-000146.txt'\n...,...,-0.047265,-0.005206,-0.013514,0.024926,0.016773,0.056768,-0.189329,0.066934,-0.024946,0.088031
3960,1018724,AMAZON COM INC,2019-12-31,[334103],334103,1,NaN,0.00,4 days 00:00:00,['edgar/data/1018724/0000899243-19-030277.txt'],...,-0.029111,0.067372,0.101078,0.069387,0.138002,0.082767,-0.100004,0.033996,0.008402,0.076832
65091,54480,KANSAS CITY SOUTHERN,2019-12-31,[827929 827930 827932 827933 827934 827935 827...,827929,10,-0.001570,-285.00,11 days 00:00:00,['edgar/data/54480/0000054480-19-000247.txt'],...,0.036840,0.001612,0.076889,0.037715,0.063520,0.046904,-0.281522,0.047828,-0.197721,0.069084


['CIK',
 'Company_Names',
 'Date_Filed',
 'index_id',
 'index_id_min',
 'n_transactions',
 'shares_percent_changes_mean',
 'transactionSharesAdjust',
 'date_ft_delta_mean',
 'Filename',
 'Form_Type',
 'conversionOrExercisePrice',
 'date_ft_delta',
 'deemedExecutionDate',
 'directOrIndirectOwnership',
 'doc_name',
 'exerciseDate',
 'expirationDate',
 'file_url',
 'isDirector',
 'isOfficer',
 'isOther',
 'isTenPercentOwner',
 'issuerName',
 'issuerTradingSymbol',
 'officerTitle',
 'otherText',
 'periodOfReport',
 'quarter',
 'rptOwnerName',
 'securityTitle',
 'sharesOwnedFollowingTransaction',
 'shares_percent_changes',
 'signatureDate',
 'transactionAcquiredDisposedCode',
 'transactionCode',
 'transactionDate',
 'transactionFormType',
 'transactionPricePerShare',
 'transactionShares',
 'transactionTotalValue',
 'transaction_table',
 'transaction_type',
 'underlyingSecurityShares',
 'underlyingSecurityTitle',
 'underlyingSecurityValue',
 'valueOwnedFollowingTransaction',
 'year',
 'my_de

## Performance Metrics Evaluations

In [6]:
pe_4form = PerformanceEvaluations4Form(processed_4form_df, n_sub_set=5)

In [7]:
days = 21
column_label_eval = 'Price pct_change ({days})'.format(days=days)
shifted_columns_label_eval = 'Shifted Price pct_change ({days})(-{days} days +00:00:00)'.format(days=days)

In [8]:

# PerformanceMetric01 evaluation:
metrics_df_01, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric01,
                                        pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation: ")
display(metrics_df_01)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_02b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation: ")
display(metrics_df_02)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_03b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation: ")
display(metrics_df_03)
print("-" * 75)


PerformanceMetric01 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.011708,0.014604,0.009405,0.011312,0.011447,0.011772
benchmark,0.010012,0.010712,0.009915,0.009609,0.009068,0.010755
is_greater_prob,0.585882,0.694118,0.529412,0.623529,0.564706,0.517647
is_pos_tse_ok,True,True,False,True,True,True
is_greater_prob_ok,True,True,True,True,True,True
is_performance_ok,True,True,False,True,True,True
Positive_tsa_std,0.015391,0.012219,0.013926,0.013213,0.014226,0.021481
benchmark_std,0.009589,0.00781,0.008747,0.00791,0.007538,0.014322
statistic,1.927937,2.474472,-0.286391,1.019083,1.362238,0.363208
pvalue,0.027131,0.007257,0.612501,0.154978,0.08776,0.358487


---------------------------------------------------------------------------

PerformanceMetric02 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.011708,0.014604,0.009405,0.011312,0.011447,0.011772
benchmark,0.00991,0.013232,0.006067,0.013705,0.008019,0.008526
is_greater_prob,0.578824,0.6,0.682353,0.470588,0.564706,0.576471
is_pos_tse_ok,True,True,True,False,True,True
is_greater_prob_ok,True,True,True,False,True,True
is_performance_ok,True,True,True,False,True,True
Positive_tsa_std,0.015391,0.012219,0.013926,0.013213,0.014226,0.021481
benchmark_std,0.020799,0.022339,0.014072,0.017115,0.017448,0.029032
statistic,1.432374,0.496728,1.55409,-1.020595,1.40348,0.82869
pvalue,0.076218,0.310109,0.061022,0.845496,0.081197,0.204278


---------------------------------------------------------------------------

PerformanceMetric03 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.011762,0.013571,0.010482,0.011083,0.011426,0.012029
benchmark,0.009798,0.012563,0.007339,0.011839,0.008279,0.007677
is_greater_prob,0.500342,0.502447,0.497716,0.481093,0.512862,0.514286
is_pos_tse_ok,True,True,True,False,True,True
is_greater_prob_ok,True,True,False,False,True,True
is_performance_ok,True,True,False,False,True,True
Positive_tsa_std,0.079716,0.085366,0.076684,0.079245,0.074738,0.080629
benchmark_std,0.103252,0.144578,0.07906,0.089056,0.082661,0.090523
statistic,2.229463,0.446101,2.025104,-0.429206,1.724958,1.992223
pvalue,0.012894,0.327768,0.021442,0.666108,0.042288,0.023196


---------------------------------------------------------------------------


In [9]:
print("=" * 90)
pe_4form_year = PerformanceEvaluations4FormYear(processed_4form_df)

# PerformanceMetric01 evaluation:
metrics_df_01_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric01,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation (Year): ")
display(metrics_df_01_year)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_02b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation (Year): ")
display(metrics_df_02_year)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_03b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation (Year): ")
display(metrics_df_03_year)
print("-" * 75)

print("=" * 90)


PerformanceMetric01 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.011708,0.019224,0.003944,0.013261,0.022478,0.009882,-0.005613,0.02685,0.013336,-0.000163,0.018719
benchmark,0.010012,0.017813,-0.001751,0.015201,0.015772,0.009634,-0.010093,0.020306,0.013429,-0.006975,0.013227
is_greater_prob,0.585882,0.553299,0.573935,0.471033,0.618454,0.521197,0.584352,0.589806,0.480769,0.589928,0.572792
is_pos_tse_ok,True,True,True,False,True,True,True,True,False,True,True
is_greater_prob_ok,True,True,True,False,True,True,True,True,False,True,True
is_performance_ok,True,True,True,False,True,True,True,True,False,True,True
Positive_tsa_std,0.015391,0.047399,0.049872,0.04046,0.042075,0.056612,0.043143,0.047826,0.036846,0.044499,0.042534
benchmark_std,0.009589,0.027845,0.02944,0.026519,0.024636,0.021373,0.024769,0.036544,0.026081,0.026436,0.027887
statistic,1.927937,0.509279,1.964116,-0.798985,2.754053,0.082042,1.821514,2.206907,-0.041999,2.687838,2.210444
pvalue,0.027131,0.305367,0.024973,0.787711,0.003026,0.467323,0.034494,0.013808,0.516744,0.003684,0.013694


---------------------------------------------------------------------------

PerformanceMetric02 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.011708,0.019224,0.003944,0.013261,0.022478,0.009882,-0.005613,0.02685,0.013336,-0.000163,0.018719
benchmark,0.00991,0.018909,0.006005,0.016775,0.024816,0.01727,0.001831,0.016701,0.013829,-0.009828,0.015458
is_greater_prob,0.578824,0.489848,0.533835,0.468514,0.483791,0.379052,0.437653,0.584951,0.473558,0.561151,0.529833
is_pos_tse_ok,True,True,False,False,False,False,False,True,False,True,True
is_greater_prob_ok,True,False,True,False,False,False,False,True,False,True,True
is_performance_ok,True,False,False,False,False,False,False,True,False,True,True
Positive_tsa_std,0.015391,0.047399,0.049872,0.04046,0.042075,0.056612,0.043143,0.047826,0.036846,0.044499,0.042534
benchmark_std,0.020799,0.051723,0.096204,0.046654,0.043123,0.063183,0.046466,0.048057,0.044343,0.050418,0.04942
statistic,1.432374,0.089148,-0.380031,-1.133732,-0.777201,-1.744039,-2.374152,3.038382,-0.174375,2.935036,1.023801
pvalue,0.076218,0.464494,0.647971,0.871372,0.781365,0.95923,0.991089,0.001227,0.569193,0.001714,0.153116


---------------------------------------------------------------------------

PerformanceMetric03 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.011762,0.020393,0.004371,0.01043,0.021512,0.012261,-0.005559,0.025723,0.012656,-0.001068,0.017438
benchmark,0.009798,0.019677,0.001272,0.01345,0.02179,0.018171,-0.001935,0.014674,0.010892,-0.010227,0.011963
is_greater_prob,0.500342,0.495553,0.516464,0.49784,0.49758,0.459071,0.469234,0.514768,0.49373,0.544353,0.508673
is_pos_tse_ok,True,True,True,False,False,False,False,True,True,True,True
is_greater_prob_ok,True,False,True,False,False,False,False,True,False,True,True
is_performance_ok,True,False,True,False,False,False,False,True,False,True,True
Positive_tsa_std,0.079716,0.088827,0.110957,0.077235,0.065005,0.064578,0.07111,0.085668,0.059508,0.077129,0.07437
benchmark_std,0.103252,0.090232,0.214369,0.080173,0.065838,0.074708,0.079884,0.085793,0.074221,0.08168,0.083235
statistic,2.229463,0.274965,0.6169,-1.238115,-0.136388,-2.735279,-1.563435,4.208635,0.860449,3.919464,2.355096
pvalue,0.012894,0.391677,0.268671,0.892128,0.554239,0.99687,0.940987,0.000013,0.194796,0.000045,0.00928


---------------------------------------------------------------------------


## OutLayers Rem Evaluation test

In [10]:
post_processed_4form_df = processed_4form_df.copy()
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df.my_derivative_types.isin(['AB', 'B'])]
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df['transactionSharesAdjust'] > 0]

In [11]:
def _ecdf(sample):
    sample = np.atleast_1d(sample)
    quantiles, counts = np.unique(sample, return_counts=True)
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob


def _get_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[np.logical_and(cumprob > 0.5 * ci, cumprob < 1. - 0.5 * ci)]

def _get_left_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[cumprob < 1. - ci]

quantiles, cumprob = _ecdf(post_processed_4form_df[column_label_eval].values)
results_dict = dict()
results_dict["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()

quantiles, cumprob = _ecdf(post_processed_4form_df[shifted_columns_label_eval].values)
results_dict_bench = dict()
results_dict_bench["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict_bench["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict_bench["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict_bench["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()

In [12]:
print("\n MostProb Scenarios results: \n")
display(pd.Series(results_dict))
print("\n MostProb Scenarios results Bench: \n")
display(pd.Series(results_dict_bench))


 MostProb Scenarios results: 



E[R|0.05<P(R)<0.95]    0.011978
E[R|0.10<P(R)<0.90]    0.012042
E[R|0.15<P(R)<0.85]    0.012159
E[R|0.20<P(R)<0.80]    0.012260
dtype: float64


 MostProb Scenarios results Bench: 



E[R|0.05<P(R)<0.95]    0.009771
E[R|0.10<P(R)<0.90]    0.010176
E[R|0.15<P(R)<0.85]    0.010541
E[R|0.20<P(R)<0.80]    0.010819
dtype: float64